# Not sure if this file is up to date - for operational usage go to Experiment_Automation.ipynb !!

# Test Environment

In [2]:
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import pandas as pd
#import Models.models as models
#import Models.LSTM.models_LSTM as models
import script_utils as mL
#from Experiment_Class import Experiment
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pickle
import os

NDBC = mL.NDBC_lib
ERA5 = mL.ERA5_lib
Models = mL.Models
DP = mL.DataProcessor
Experiment = mL.Experiment

# %load_ext jupyternotify

# Get Data

In [3]:
data_directory = os.path.join(os.getcwd(), f'data/datasets/type_A')

def build_UI():

    # Select Model-------------------------------------------------------------------------------
    global datafile_widget
    datafile_list = os.listdir(data_directory)
    datafile_widget = widgets.Select(
        options=datafile_list,
        value=datafile_list[0],
        # rows=10,
        description='Datafile:',
        disabled=False
    )

    #STATIONARY_SHIFT -----------------------------------------------------------------------
    global stationaryShift_widget
    stationaryShift_widget = widgets.BoundedIntText(
        value=1,
        min=0,
        max=10,
        step=1,
        description='',
        disabled=False,
    )

    #Supervised:
    n_in_dropdown = widgets.Dropdown(
        options=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        value=3,
        description='n_in:',
        disabled=False,
    )
    n_out_dropdown = widgets.Dropdown(
        options=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        value=1,
        description='n_out:',
        disabled=False,
    )

    # Test Hours-------------------------------------------------------------------------------
    global test_hours_widget
    test_hours_widget = widgets.IntSlider(
        value=24,
        min=0,
        max=1000,
        step=1,
        description='Test Hours:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d'
    )

    # Select Model-------------------------------------------------------------------------------
    global models_widget
    models_list = list(Models.model_dictionary.keys())
    models_widget = widgets.Select(
        options=models_list,
        value=models_list[0],
        rows=10,
        description='Model:',
        disabled=False
    )

    #ALPHA:-------------------------------------------------------------------------------
    # create a FloatSlider widget for a value between 0 and 1
    global alpha_slider
    alpha_slider = widgets.FloatSlider(
        value=0.5,
        min=0,
        max=1,
        step=0.01,
        description='',
        readout_format='.2f',
        orientation='horizontal',
        layout={'width': '500px'}
    )

    left_box = widgets.VBox([
        datafile_widget,
        n_in_dropdown,
        n_out_dropdown,
        widgets.HTML(value="<p>Station Shift:</p>"),
        stationaryShift_widget,
        test_hours_widget
        ],
    )


    right_box = widgets.VBox([
        models_widget
    ])

    display(widgets.HBox([
        left_box,
        right_box,
    ]))
    print("Alpha: ")
    display(alpha_slider)

build_UI() #can be activated whenever needed!

Alpha: 


FloatSlider(value=0.5, layout=Layout(width='500px'), max=1.0, step=0.01)

In [4]:
# DATAFILE = datafile_widget.value
# STATIONARY_SHIFT = stationaryShift_widget.value
# N_TEST_HOURS = test_hours_widget.value
# MODEL_NAME = models_widget.value
# ALPHA = alpha_slider.value

## Optional: use hardcoded variables instead

In [5]:
DATAFILE = "42003_stage_2.pickle"
N_IN=24
N_OUT=24
#STATIONARY_SHIFT = 1
N_TEST_HOURS = 31*24 #(One month)
MODEL_NAME = "LSTM_3D"
ALPHA = 1

In [6]:
print(f"Datafile: {DATAFILE}")
print(f"Alpha: {ALPHA}")
#print(f"Stationary Shift: {STATIONARY_SHIFT}")
print(f"N_IN: {N_IN}")
print(f"N_OUT: {N_OUT}")
print(f"Test-Hours: {N_TEST_HOURS}")
print(f"Model: {MODEL_NAME}")

Datafile: 42003_stage_2.pickle
Alpha: 1
N_IN: 24
N_OUT: 24
Test-Hours: 744
Model: LSTM_3D


In [7]:
with open(f'data/datasets/type_A/{DATAFILE}', 'rb') as f:
    # load the object from the file using pickle.load()
    dataset = pickle.load(f)

print("stations",dataset["stations"])
print("years",dataset["years"])
print("nan_threshold",dataset["nan_threshold"])
print("features",dataset["features"])
print("add_era5",dataset["add_era5"])

data = dataset["data"]
data

stations ['42040', '42038', '42007', '42001', '42002', '42003', '42012', '42019', '42035', '42036', '42039', '41041', '42055']
years ['2017', '2018', '2019', '2020', '2021', '2022']
nan_threshold 0.66
features ['WDIR', 'WSPD', 'WVHT', 'APD', 'MWD', 'PRES', 'ATMP', 'WTMP', 'DEWP']
add_era5 True


,WDIR_42001,WSPD_42001,PRES_42001,ATMP_42001,WTMP_42001,DEWP_42001,WDIR_42002,WSPD_42002,PRES_42002,ATMP_42002,...,DEWP_42002_ERA5,WDIR_42039_ERA5,WSPD_42039_ERA5,ATMP_42039_ERA5,WSPD_42035_ERA5,WSPD_42001_ERA5,ATMP_42019_ERA5,WTMP_42039_ERA5,WSPD_42002_ERA5,PRES_42039_ERA5
2017-01-01 00:00:00,107.0,8.1,1013.2,26.6,29.8,23.8,53.0,2.9,1016.4,23.3,...,21.860131,91.620952,8.348397,22.496642,4.389762,7.304383,22.738986,23.406143,7.976832,1017.816021
2017-01-01 01:00:00,107.0,8.1,1013.2,26.6,29.8,23.8,53.0,2.9,1016.4,23.3,...,21.885986,91.614961,8.460653,22.854421,4.734988,7.045226,22.988974,23.406143,7.374513,1018.317071
2017-01-01 02:00:00,107.0,8.1,1013.2,26.6,29.8,23.8,53.0,2.9,1016.4,23.3,...,21.926532,90.990649,7.987505,23.165863,4.716014,7.172287,23.073993,23.406143,7.036900,1018.731352
2017-01-01 03:00:00,107.0,8.1,1013.2,26.6,29.8,23.8,53.0,2.9,1016.4,23.3,...,21.989702,93.380226,7.735322,23.488559,4.520726,7.446157,23.121183,23.406143,6.923938,1018.723523
2017-01-01 04:00:00,107.0,8.1,1013.2,26.6,29.8,23.8,53.0,2.9,1016.4,23.3,...,22.104876,99.283786,7.698163,23.722886,4.060894,7.788665,23.175782,23.406143,7.397256,1018.085466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31 19:00:00,195.0,0.5,1015.0,25.6,24.7,24.5,22.0,0.6,1015.9,25.1,...,20.512644,28.180851,8.602918,22.730194,2.533178,0.601025,21.847184,24.392457,1.062895,1016.581840
2022-12-31 20:00:00,210.0,1.0,1015.0,25.3,24.5,24.3,88.0,1.2,1015.4,24.7,...,20.593026,22.969408,7.734290,22.564177,2.669909,0.979714,22.052563,24.392457,1.330008,1016.363260
2022-12-31 21:00:00,231.0,0.6,1014.7,26.4,24.7,24.4,87.0,1.7,1014.9,24.6,...,20.705755,14.655430,6.485218,22.377552,3.147255,2.067327,22.260412,24.392457,1.770915,1016.186435
2022-12-31 22:00:00,18.0,0.7,1014.9,25.4,24.7,24.1,90.0,2.5,1014.9,24.5,...,20.222161,20.371840,3.294807,24.068445,3.469271,1.554299,22.101541,24.885927,2.483432,1017.067575


# Data Processing

## 1. Make data stationary

Austin mentioned that it might be worth it skipping this step to get more convincing results.

## 2. Transform to supervised problem

In [205]:
#data_supervised = DP.data_to_supervised(data_stationary, n_in=N_IN, n_out=N_OUT)
data_supervised = DP.data_to_supervised(data, n_in=N_IN, n_out=N_OUT)
data_supervised.head()

,WDIR_42001(t-24),WSPD_42001(t-24),PRES_42001(t-24),ATMP_42001(t-24),WTMP_42001(t-24),DEWP_42001(t-24),WDIR_42002(t-24),WSPD_42002(t-24),PRES_42002(t-24),ATMP_42002(t-24),...,DEWP_42002_ERA5(t+23),WDIR_42039_ERA5(t+23),WSPD_42039_ERA5(t+23),ATMP_42039_ERA5(t+23),WSPD_42035_ERA5(t+23),WSPD_42001_ERA5(t+23),ATMP_42019_ERA5(t+23),WTMP_42039_ERA5(t+23),WSPD_42002_ERA5(t+23),PRES_42039_ERA5(t+23)
2017-01-02 00:00:00,107.0,8.1,1013.2,26.6,29.8,23.8,53.0,2.9,1016.4,23.3,...,22.712186,100.758006,5.600168,24.089601,3.027370,6.414373,22.095493,23.516289,5.447760,1014.172311
2017-01-02 01:00:00,107.0,8.1,1013.2,26.6,29.8,23.8,53.0,2.9,1016.4,23.3,...,22.767129,112.998793,7.059138,24.074376,3.468129,6.809022,22.109533,23.516289,5.496215,1014.978038
2017-01-02 02:00:00,107.0,8.1,1013.2,26.6,29.8,23.8,53.0,2.9,1016.4,23.3,...,22.838819,109.324571,8.449554,24.029364,3.731152,7.015621,22.160232,23.516289,4.857116,1015.098081
2017-01-02 03:00:00,107.0,8.1,1013.2,26.6,29.8,23.8,53.0,2.9,1016.4,23.3,...,22.878190,94.179325,8.861126,24.063785,3.605799,6.296834,22.208202,23.516289,3.256610,1016.216312
2017-01-02 04:00:00,107.0,8.1,1013.2,26.6,29.8,23.8,53.0,2.9,1016.4,23.3,...,22.801505,79.816156,8.779075,24.014801,3.653944,4.929976,22.280351,23.516289,2.170185,1016.259371


## train test split

In [206]:
def train_test_split_3d(df, num_test_records):
    # Extract the unique timestamps for X and y
    x_timestamps = []
    y_timestamps = []

    for col in df.columns:
        timestamp = col.split('(')[-1][:-1]
        if 't-' in col and timestamp not in x_timestamps:
            x_timestamps.append(timestamp)
        if ('t+' in col or timestamp == 't') and timestamp not in y_timestamps:
            y_timestamps.append(timestamp)

    # Number of features per timestamp
    num_x_features = len(df.columns) // (len(x_timestamps) + len(y_timestamps))
    num_y_features = num_x_features

    # Initialize 3D arrays to hold the data
    num_records = len(df)
    x_data = np.zeros((num_records, len(x_timestamps), num_x_features))
    y_data = np.zeros((num_records, len(y_timestamps), num_y_features))

    for i, timestamp in enumerate(x_timestamps):
        cols = [col for col in df.columns if f'({timestamp})' in col]
        x_data[:, i, :] = df[cols].values

    for i, timestamp in enumerate(y_timestamps):
        cols = [col for col in df.columns if f'({timestamp})' in col]
        y_data[:, i, :] = df[cols].values

    # Split the data into train and test sets
    train_x_data = x_data[:-num_test_records]
    test_x_data = x_data[-num_test_records:]
    train_y_data = y_data[:-num_test_records]
    test_y_data = y_data[-num_test_records:]

    # Create train_X, train_y, test_X, and test_y
    train_X = train_x_data
    train_y = train_y_data
    test_X = test_x_data
    test_y = test_y_data

    y_headers = [col for col in data_supervised.columns if ('t+' in col or '(t)' in col) and 't-' not in col]

    return train_X, train_y, test_X, test_y, y_headers

def y_to_dataframe(y_array, y_headers):
    records, timestamps, features = y_array.shape
    data = y_array.reshape(records, timestamps * features)
    return pd.DataFrame(data, columns=y_headers)

In [207]:
train_X, train_y, test_X, test_y, y_headers = train_test_split_3d(data_supervised, N_TEST_HOURS)
print("Shapes: ", train_X.shape, train_y.shape, test_X.shape, test_y.shape)

Shapes:  (51793, 24, 92) (51793, 24, 92) (744, 24, 92) (744, 24, 92)


In [208]:
#DEV Data:
# Reducing the data to 10 records for development purposes
#train_X, train_y, test_X, test_y = train_X[:10], train_y[:10], test_X[:10], test_y[:10]
#print("Shapes: ", train_X.shape, train_y.shape, test_X.shape, test_y.shape)

## Train Mdoel

In [209]:
model = Models.get_model(MODEL_NAME, train_X, train_y, ALPHA)

Epoch 1/100
729/729 [==============================] - 39s 50ms/step - loss: 206.3045 - val_loss: 204.4026
Epoch 2/100
729/729 [==============================] - 37s 51ms/step - loss: 196.9839 - val_loss: 196.2111
Epoch 3/100
729/729 [==============================] - 32s 44ms/step - loss: 191.6054 - val_loss: 191.8206
Epoch 4/100
729/729 [==============================] - 32s 43ms/step - loss: 187.6810 - val_loss: 187.9917
Epoch 5/100
729/729 [==============================] - 31s 43ms/step - loss: 183.9438 - val_loss: 184.5692
Epoch 6/100
729/729 [==============================] - 31s 42ms/step - loss: 180.6688 - val_loss: 181.6256
Epoch 7/100
729/729 [==============================] - 31s 43ms/step - loss: 177.6433 - val_loss: 178.8482
Epoch 8/100
729/729 [==============================] - 32s 43ms/step - loss: 174.7685 - val_loss: 176.1530
Epoch 9/100
729/729 [==============================] - 31s 42ms/step - loss: 172.1317 - val_loss: 173.7199
Epoch 10/100
729/729 [===============

# CONTINUE HERE

In [211]:
print('Ready to continue')

Ready to continue


----
# One-Shot Forecasting

In [212]:
_ = model.predict(train_X, batch_size=1)

51793/51793 [==============================] - 144s 3ms/step


In [213]:
#output_cols = data.columns.tolist() #??
#output_cols

In [214]:
yhat = model.predict(test_X)

24/24 [==============================] - 1s 8ms/step


In [215]:
header_results=[name + "_pred" for name in y_headers]
yhat_df = y_to_dataframe(yhat, header_results)
yhat_df.set_index(data_supervised.tail(len(yhat)).index, inplace=True)
yhat_df

,WDIR_42001(t)_pred,WSPD_42001(t)_pred,PRES_42001(t)_pred,ATMP_42001(t)_pred,WTMP_42001(t)_pred,DEWP_42001(t)_pred,WDIR_42002(t)_pred,WSPD_42002(t)_pred,PRES_42002(t)_pred,ATMP_42002(t)_pred,...,DEWP_42002_ERA5(t+23)_pred,WDIR_42039_ERA5(t+23)_pred,WSPD_42039_ERA5(t+23)_pred,ATMP_42039_ERA5(t+23)_pred,WSPD_42035_ERA5(t+23)_pred,WSPD_42001_ERA5(t+23)_pred,ATMP_42019_ERA5(t+23)_pred,WTMP_42039_ERA5(t+23)_pred,WSPD_42002_ERA5(t+23)_pred,PRES_42039_ERA5(t+23)_pred
2022-11-30 01:00:00,100.951622,3.470601,787.958862,22.877684,22.515289,20.328766,45.217014,4.466831,788.635681,19.07692,...,0.968116,-0.02937,-0.741868,0.03353,0.027804,-0.17851,0.425472,0.065564,0.103171,0.129197
2022-11-30 02:00:00,100.951622,3.470601,787.958862,22.877684,22.515289,20.328766,45.217014,4.466831,788.635681,19.07692,...,0.968116,-0.02937,-0.741868,0.03353,0.027804,-0.17851,0.425472,0.065564,0.103171,0.129197
2022-11-30 03:00:00,100.951622,3.470601,787.958862,22.877684,22.515289,20.328766,45.217014,4.466831,788.635681,19.07692,...,0.968116,-0.02937,-0.741868,0.03353,0.027804,-0.17851,0.425472,0.065564,0.103171,0.129197
2022-11-30 04:00:00,100.951622,3.470601,787.958862,22.877684,22.515289,20.328766,45.217014,4.466831,788.635681,19.07692,...,0.968116,-0.02937,-0.741868,0.03353,0.027804,-0.17851,0.425472,0.065564,0.103171,0.129197
2022-11-30 05:00:00,100.951622,3.470601,787.958862,22.877684,22.515289,20.328766,45.217014,4.466831,788.635681,19.07692,...,0.968116,-0.02937,-0.741868,0.03353,0.027804,-0.17851,0.425472,0.065564,0.103171,0.129197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-30 20:00:00,100.951622,3.470601,787.958862,22.877684,22.515289,20.328766,45.217014,4.466831,788.635681,19.07692,...,0.968116,-0.02937,-0.741868,0.03353,0.027804,-0.17851,0.425472,0.065564,0.103171,0.129197
2022-12-30 21:00:00,100.951622,3.470601,787.958862,22.877684,22.515289,20.328766,45.217014,4.466831,788.635681,19.07692,...,0.968116,-0.02937,-0.741868,0.03353,0.027804,-0.17851,0.425472,0.065564,0.103171,0.129197
2022-12-30 22:00:00,100.951622,3.470601,787.958862,22.877684,22.515289,20.328766,45.217014,4.466831,788.635681,19.07692,...,0.968116,-0.02937,-0.741868,0.03353,0.027804,-0.17851,0.425472,0.065564,0.103171,0.129197
2022-12-30 23:00:00,100.951622,3.470601,787.958862,22.877684,22.515289,20.328766,45.217014,4.466831,788.635681,19.07692,...,0.968116,-0.02937,-0.741868,0.03353,0.027804,-0.17851,0.425472,0.065564,0.103171,0.129197


In [216]:
evaluation = data_supervised[y_headers].tail(N_TEST_HOURS)
evaluation

,WDIR_42001(t),WSPD_42001(t),PRES_42001(t),ATMP_42001(t),WTMP_42001(t),DEWP_42001(t),WDIR_42002(t),WSPD_42002(t),PRES_42002(t),ATMP_42002(t),...,DEWP_42002_ERA5(t+23),WDIR_42039_ERA5(t+23),WSPD_42039_ERA5(t+23),ATMP_42039_ERA5(t+23),WSPD_42035_ERA5(t+23),WSPD_42001_ERA5(t+23),ATMP_42019_ERA5(t+23),WTMP_42039_ERA5(t+23),WSPD_42002_ERA5(t+23),PRES_42039_ERA5(t+23)
2022-11-30 01:00:00,143.0,5.1,1013.5,26.0,25.8,26.1,169.0,7.1,1012.2,26.4,...,22.305537,242.227408,7.923812,22.793180,11.096199,6.853869,18.660720,24.087488,10.980621,1020.541856
2022-11-30 02:00:00,146.0,5.5,1014.1,26.0,25.7,26.1,168.0,7.9,1012.8,26.4,...,22.293447,239.596934,7.981794,22.885326,11.519451,7.894913,18.769789,24.087488,11.127536,1021.053156
2022-11-30 03:00:00,145.0,5.8,1014.4,26.1,25.7,26.1,165.0,8.7,1012.9,26.3,...,22.082364,242.366288,8.232870,22.472808,11.726001,8.438319,18.818356,24.087488,11.483298,1021.884444
2022-11-30 04:00:00,151.0,5.0,1014.7,26.0,25.7,26.0,168.0,7.3,1013.6,26.4,...,21.808872,247.323459,8.985751,21.507806,11.921091,8.827513,18.804774,24.087488,11.712454,1022.180572
2022-11-30 05:00:00,155.0,4.9,1015.0,26.0,25.6,26.0,174.0,6.5,1014.2,26.3,...,21.497804,249.279171,9.954728,20.622119,11.674992,9.112413,18.735216,24.087488,11.972046,1022.342483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-30 20:00:00,174.0,8.3,1012.1,25.4,24.1,24.3,268.0,7.7,1011.1,25.2,...,20.512644,28.180851,8.602918,22.730194,2.533178,0.601025,21.847184,24.392457,1.062895,1016.581840
2022-12-30 21:00:00,178.0,7.1,1012.1,25.3,24.1,24.5,277.0,6.6,1011.1,25.3,...,20.593026,22.969408,7.734290,22.564177,2.669909,0.979714,22.052563,24.392457,1.330008,1016.363260
2022-12-30 22:00:00,185.0,7.1,1012.5,25.4,24.2,24.2,315.0,6.5,1011.3,24.6,...,20.705755,14.655430,6.485218,22.377552,3.147255,2.067327,22.260412,24.392457,1.770915,1016.186435
2022-12-30 23:00:00,185.0,5.9,1012.9,25.3,24.2,24.4,322.0,5.5,1011.7,24.7,...,20.222161,20.371840,3.294807,24.068445,3.469271,1.554299,22.101541,24.885927,2.483432,1017.067575


In [217]:
evaluation = pd.concat([evaluation, yhat_df], axis=1)
evaluation

,WDIR_42001(t),WSPD_42001(t),PRES_42001(t),ATMP_42001(t),WTMP_42001(t),DEWP_42001(t),WDIR_42002(t),WSPD_42002(t),PRES_42002(t),ATMP_42002(t),...,DEWP_42002_ERA5(t+23)_pred,WDIR_42039_ERA5(t+23)_pred,WSPD_42039_ERA5(t+23)_pred,ATMP_42039_ERA5(t+23)_pred,WSPD_42035_ERA5(t+23)_pred,WSPD_42001_ERA5(t+23)_pred,ATMP_42019_ERA5(t+23)_pred,WTMP_42039_ERA5(t+23)_pred,WSPD_42002_ERA5(t+23)_pred,PRES_42039_ERA5(t+23)_pred
2022-11-30 01:00:00,143.0,5.1,1013.5,26.0,25.8,26.1,169.0,7.1,1012.2,26.4,...,0.968116,-0.02937,-0.741868,0.03353,0.027804,-0.17851,0.425472,0.065564,0.103171,0.129197
2022-11-30 02:00:00,146.0,5.5,1014.1,26.0,25.7,26.1,168.0,7.9,1012.8,26.4,...,0.968116,-0.02937,-0.741868,0.03353,0.027804,-0.17851,0.425472,0.065564,0.103171,0.129197
2022-11-30 03:00:00,145.0,5.8,1014.4,26.1,25.7,26.1,165.0,8.7,1012.9,26.3,...,0.968116,-0.02937,-0.741868,0.03353,0.027804,-0.17851,0.425472,0.065564,0.103171,0.129197
2022-11-30 04:00:00,151.0,5.0,1014.7,26.0,25.7,26.0,168.0,7.3,1013.6,26.4,...,0.968116,-0.02937,-0.741868,0.03353,0.027804,-0.17851,0.425472,0.065564,0.103171,0.129197
2022-11-30 05:00:00,155.0,4.9,1015.0,26.0,25.6,26.0,174.0,6.5,1014.2,26.3,...,0.968116,-0.02937,-0.741868,0.03353,0.027804,-0.17851,0.425472,0.065564,0.103171,0.129197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-30 20:00:00,174.0,8.3,1012.1,25.4,24.1,24.3,268.0,7.7,1011.1,25.2,...,0.968116,-0.02937,-0.741868,0.03353,0.027804,-0.17851,0.425472,0.065564,0.103171,0.129197
2022-12-30 21:00:00,178.0,7.1,1012.1,25.3,24.1,24.5,277.0,6.6,1011.1,25.3,...,0.968116,-0.02937,-0.741868,0.03353,0.027804,-0.17851,0.425472,0.065564,0.103171,0.129197
2022-12-30 22:00:00,185.0,7.1,1012.5,25.4,24.2,24.2,315.0,6.5,1011.3,24.6,...,0.968116,-0.02937,-0.741868,0.03353,0.027804,-0.17851,0.425472,0.065564,0.103171,0.129197
2022-12-30 23:00:00,185.0,5.9,1012.9,25.3,24.2,24.4,322.0,5.5,1011.7,24.7,...,0.968116,-0.02937,-0.741868,0.03353,0.027804,-0.17851,0.425472,0.065564,0.103171,0.129197


# Correct wind direction (modulo 360)

In [218]:
# Get the list of columns starting with "WDIR"
wdir_columns = [col for col in evaluation.columns if col.startswith("WDIR")]

# Modify the values in the selected columns
evaluation[wdir_columns] = evaluation[wdir_columns] % 360

evaluation

,WDIR_42001(t),WSPD_42001(t),PRES_42001(t),ATMP_42001(t),WTMP_42001(t),DEWP_42001(t),WDIR_42002(t),WSPD_42002(t),PRES_42002(t),ATMP_42002(t),...,DEWP_42002_ERA5(t+23)_pred,WDIR_42039_ERA5(t+23)_pred,WSPD_42039_ERA5(t+23)_pred,ATMP_42039_ERA5(t+23)_pred,WSPD_42035_ERA5(t+23)_pred,WSPD_42001_ERA5(t+23)_pred,ATMP_42019_ERA5(t+23)_pred,WTMP_42039_ERA5(t+23)_pred,WSPD_42002_ERA5(t+23)_pred,PRES_42039_ERA5(t+23)_pred
2022-11-30 01:00:00,143.0,5.1,1013.5,26.0,25.8,26.1,169.0,7.1,1012.2,26.4,...,0.968116,359.970642,-0.741868,0.03353,0.027804,-0.17851,0.425472,0.065564,0.103171,0.129197
2022-11-30 02:00:00,146.0,5.5,1014.1,26.0,25.7,26.1,168.0,7.9,1012.8,26.4,...,0.968116,359.970642,-0.741868,0.03353,0.027804,-0.17851,0.425472,0.065564,0.103171,0.129197
2022-11-30 03:00:00,145.0,5.8,1014.4,26.1,25.7,26.1,165.0,8.7,1012.9,26.3,...,0.968116,359.970642,-0.741868,0.03353,0.027804,-0.17851,0.425472,0.065564,0.103171,0.129197
2022-11-30 04:00:00,151.0,5.0,1014.7,26.0,25.7,26.0,168.0,7.3,1013.6,26.4,...,0.968116,359.970642,-0.741868,0.03353,0.027804,-0.17851,0.425472,0.065564,0.103171,0.129197
2022-11-30 05:00:00,155.0,4.9,1015.0,26.0,25.6,26.0,174.0,6.5,1014.2,26.3,...,0.968116,359.970642,-0.741868,0.03353,0.027804,-0.17851,0.425472,0.065564,0.103171,0.129197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-30 20:00:00,174.0,8.3,1012.1,25.4,24.1,24.3,268.0,7.7,1011.1,25.2,...,0.968116,359.970642,-0.741868,0.03353,0.027804,-0.17851,0.425472,0.065564,0.103171,0.129197
2022-12-30 21:00:00,178.0,7.1,1012.1,25.3,24.1,24.5,277.0,6.6,1011.1,25.3,...,0.968116,359.970642,-0.741868,0.03353,0.027804,-0.17851,0.425472,0.065564,0.103171,0.129197
2022-12-30 22:00:00,185.0,7.1,1012.5,25.4,24.2,24.2,315.0,6.5,1011.3,24.6,...,0.968116,359.970642,-0.741868,0.03353,0.027804,-0.17851,0.425472,0.065564,0.103171,0.129197
2022-12-30 23:00:00,185.0,5.9,1012.9,25.3,24.2,24.4,322.0,5.5,1011.7,24.7,...,0.968116,359.970642,-0.741868,0.03353,0.027804,-0.17851,0.425472,0.065564,0.103171,0.129197


# Recurrent forecast (EXCLUDED FOR NOW!)

# SAVE

In [219]:
%%notify -m "Finished!!"
print("reached checkpoint")

UsageError: Cell magic `%%notify` not found.


In [220]:
# create a text input widget for username
filename_widget = widgets.Text(
    value='',
    placeholder='Enter filename',
    description='Filename:',
    disabled=False
)
# add '.csv' to the description
extension_label = widgets.Label('.pickle')

# display the widget
display(widgets.HBox([filename_widget, extension_label]))

print("Please also check if the reports description needs to be changed!")


Please also check if the reports description needs to be changed!


In [221]:
report_description="Second LSTM experiment for publication"

In [222]:
filename = filename_widget.value
if filename == "":
    print("Enter a valid filename!")

else:
    #Save Data About executed Test:

    # Convert model summary to string
    stringlist = []
    model.summary(print_fn=lambda x: stringlist.append(x))
    model_summary = "\n".join(stringlist)

    report = Experiment(
        name=filename,
        description=report_description,

        stations = dataset["stations"],
        years = dataset["years"],
        nan_threshold=dataset["nan_threshold"],
        features=dataset["features"],
        era5=dataset["add_era5"],

        stationary_shift=None,

        n_test_hours=N_TEST_HOURS,

        #stationary=STATIONARY,
        scaler= None, # SCALER,

        model_name = MODEL_NAME,
        model_summary=model_summary,

        one_shot_forecast = evaluation,
        recursive_forecast = None,   # evaluation_2
        training_time = 0.0
    )


    # open a file for writing in binary mode
    filepath = f'data/reports/publication/{report.name}.pickle'
    with open(filepath, 'wb') as f:
        # write the object to the file using pickle.dump()
        pickle.dump(report, f)
        print("File successfully saved:")
        print(filepath)

File successfully saved:
data/reports/publication/report_42003_stage2_LSTM_1.pickle


# Create Report

TODO:
Currently the output only represents a single column !!


In [287]:
filename = "report_42003_stage0_LSTM_2"
filepath = f"data/reports/publication/{filename}.pickle"

In [288]:
import re

# open the file for reading in binary mode
report: Experiment
with open(filepath, 'rb') as f:
    # load the object from the file using pickle.load()
    report = pickle.load(f)

evaluation =report.one_shot_forecast

FileNotFoundError: [Errno 2] No such file or directory: 'data/reports/publication/report_42003_stage0_LSTM_2.pickle'

In [ ]:
def evaluate(df):
    # Initialize empty dictionaries to store MAE and MSE
    mae_dict = {}
    mse_dict = {}

    # Loop through each column and calculate MAE and MSE
    for col in df.columns:
        if col.endswith('_pred'):
            original_col = col.replace('_pred', '')
            if original_col in df.columns:
                mae_dict[original_col] = mean_absolute_error(df[original_col], df[col])
                mse_dict[original_col] = mean_squared_error(df[original_col], df[col])

    # Convert dictionaries to DataFrames
    mae_df = pd.DataFrame(list(mae_dict.items()), columns=['Title', 'Error'])
    mse_df = pd.DataFrame(list(mse_dict.items()), columns=['Title', 'Error'])

    return mae_df, mse_df

In [289]:
# def reshape_dataframe(input_df):
#     # Extract feature, timestamp from Column
#     def extract_feature_timestamp(col_name):
#         match = re.match(r'(.*)_\d+(_ERA5)?\((t\+?\d*)\)', col_name)
#         if match:
#             feature = match.group(1) + (match.group(2) if match.group(2) else '')
#             timestamp = match.group(3)
#             return feature, timestamp
#         return None, None
#
#     # Initialize a dictionary to hold the reshaped data
#     reshaped_data = {}
#
#     for _, row in input_df.iterrows():
#         feature, timestamp = extract_feature_timestamp(row['Column'])
#         if feature and timestamp:
#             if timestamp not in reshaped_data:
#                 reshaped_data[timestamp] = {}
#             reshaped_data[timestamp][feature] = row['Error']
#
#     # Convert the reshaped dictionary to a DataFrame
#     reshaped_df = pd.DataFrame.from_dict(reshaped_data, orient='index')
#
#     # Sort the DataFrame columns and index for better readability
#     reshaped_df = reshaped_df.sort_index().sort_index(axis=1)
#
#     return reshaped_df

In [290]:
def reshape_df(df):
    # Convert the input data to a DataFrame
    #df = pd.DataFrame(data, columns=['title', 'Error'])

    # Function to extract feature, stationID, origin, and timestamp from the title
    def extract_info(title):
        match = re.match(r'([A-Z]+)_(\d+)(_ERA5)?\(t([+-]?\d*)\)', title)
        if match:
            feature = match.group(1)
            stationID = match.group(2)
            origin = 'ERA5' if match.group(3) else ''
            timestamp = f't{match.group(4)}'
            return feature, stationID, origin, timestamp
        return None, None, None, None

    # Apply the extraction function to each title
    df[['Feature', 'StationID', 'Origin', 'Timestamp']] = df['Title'].apply(lambda x: pd.Series(extract_info(x)))

    # Drop the original title column
    df.drop(columns=['Title'], inplace=True)

    # Pivot the table to create the desired representation
    pivot_df = df.pivot_table(index='Timestamp', columns=['Feature', 'StationID', 'Origin'], values='Error', aggfunc='first')

    # Flatten the MultiIndex columns
    pivot_df.columns = ['_'.join(col).strip() for col in pivot_df.columns.values]

    # Reset index to have Timestamp as a column
    #pivot_df.reset_index(inplace=True)

    return pivot_df

In [291]:
mae_df, mse_df = evaluate(evaluation)
mae_df

,Title,Error
0,WDIR_42003(t),19.213348
1,WSPD_42003(t),1.030244
2,PRES_42003(t),222.370508
3,ATMP_42003(t),9.344201
4,WTMP_42003(t),7.618643
...,...,...
283,DEWP_42003_ERA5(t+23),18.054054
284,ATMP_42003_ERA5(t+23),24.320036
285,WDIR_42003_ERA5(t+23),172.051036
286,WTMP_42003_ERA5(t+23),26.051864


In [292]:
mae = reshape_df(mae_df)
mae

,ATMP_42003_,ATMP_42003_ERA5,DEWP_42003_,DEWP_42003_ERA5,PRES_42003_,PRES_42003_ERA5,WDIR_42003_,WDIR_42003_ERA5,WSPD_42003_,WSPD_42003_ERA5,WTMP_42003_,WTMP_42003_ERA5
Timestamp,,,,,,,,,,,,
t,9.344201,24.025345,8.327833,17.967124,222.370508,1018.938351,19.213348,171.156703,1.030244,4.743891,7.618643,26.217926
t+1,4.753719,24.300987,4.279012,18.061857,28.373499,1018.782196,37.146347,171.014375,2.085772,4.300367,2.420746,26.088516
t+10,3.937995,24.336540,3.559538,18.067411,6.099585,1018.763046,40.333031,171.477718,2.273337,4.203817,1.497090,26.059770
t+11,3.937995,24.335071,3.559538,18.065916,6.099585,1018.763557,40.333031,171.524873,2.273337,4.202508,1.497090,26.059118
t+12,3.937995,24.333729,3.559538,18.064583,6.099585,1018.764670,40.333031,171.588308,2.273337,4.203063,1.497090,26.058466
t+13,3.937995,24.332661,3.559538,18.063385,6.099585,1018.766313,40.333031,171.672062,2.273337,4.204350,1.497090,26.057815
t+14,3.937995,24.331688,3.559538,18.062253,6.099585,1018.767342,40.333031,171.760065,2.273337,4.204254,1.497090,26.057163
t+15,3.937995,24.331033,3.559538,18.061593,6.099585,1018.768228,40.333031,171.815786,2.273337,4.204356,1.497090,26.056511
t+16,3.937995,24.330397,3.559538,18.060980,6.099585,1018.768984,40.333031,171.859628,2.273337,4.206036,1.497090,26.055859


In [293]:
mse = reshape_df(mse_df)
mse

,ATMP_42003_,ATMP_42003_ERA5,DEWP_42003_,DEWP_42003_ERA5,PRES_42003_,PRES_42003_ERA5,WDIR_42003_,WDIR_42003_ERA5,WSPD_42003_,WSPD_42003_ERA5,WTMP_42003_,WTMP_42003_ERA5
Timestamp,,,,,,,,,,,,
t,87.314098,583.938995,69.352799,334.355538,49448.642745,1.038248e+06,369.152756,32406.143627,1.061404,28.049433,58.043724,687.683324
t+1,22.597840,597.256828,18.309946,337.761494,805.055419,1.037929e+06,1379.851099,32362.396251,4.350445,23.909206,5.860012,680.914907
t+10,15.507806,598.928255,12.670307,337.855716,37.204937,1.037890e+06,1626.753366,32593.422919,5.168063,23.048719,2.241278,679.420681
t+11,15.507806,598.850102,12.670307,337.786739,37.204937,1.037891e+06,1626.753366,32617.416293,5.168063,23.039883,2.241278,679.387245
t+12,15.507806,598.778455,12.670307,337.725077,37.204937,1.037894e+06,1626.753366,32651.116641,5.168063,23.044288,2.241278,679.353808
t+13,15.507806,598.721109,12.670307,337.669520,37.204937,1.037897e+06,1626.753366,32697.275875,5.168063,23.055178,2.241278,679.320371
t+14,15.507806,598.668725,12.670307,337.617004,37.204937,1.037899e+06,1626.753366,32746.621771,5.168063,23.054527,2.241278,679.286934
t+15,15.507806,598.633207,12.670307,337.586043,37.204937,1.037901e+06,1626.753366,32776.967830,5.168063,23.055102,2.241278,679.253497
t+16,15.507806,598.598661,12.670307,337.557201,37.204937,1.037903e+06,1626.753366,32800.626468,5.168063,23.063970,2.241278,679.220060


In [294]:
# Save DataFrames to CSV (or any other format if needed)
evaluation_path = "data/evaluation/Publication/center_42003"
mae_df.to_csv(f'{evaluation_path}/{filename}mae_results.csv', index=False)
mse_df.to_csv(f'{evaluation_path}/{filename}mse_results.csv', index=False)

print(f"Saved to: {evaluation_path}/{filename}mae_results.csv")
print(f"Saved to: {evaluation_path}/{filename}mse_results.csv")

Saved to: data/evaluation/Publication/center_42003/report_42003_stage0_LSTM_2mae_results.csv
Saved to: data/evaluation/Publication/center_42003/report_42003_stage0_LSTM_2mse_results.csv
